In [2]:
import sys 
import logging
import time
sys.path.append('..')
import warnings
warnings.simplefilter('ignore')
import pickle
import string
import os 
import numpy as np

In [3]:
research_dir = '/Users/isaachenrion/x/research/'

local_dir = os.path.join(research_dir, 'graphs')
remote_dir = os.path.join(research_dir, 'remote')
root_dir = local_dir

root_data_dir = os.path.join(root_dir, 'data/proteins')
casp11 = os.path.join(root_data_dir, 'casp11')
pdb25 = os.path.join(root_data_dir, 'pdb25')

data_dir = pdb25

### Download the pdbs from the PDB website using the id lists

In [ ]:
import urllib.request

In [6]:
def download_pdbs(id_list_filename, savedir):
    
    with open(id_list_filename, 'r') as f:
        ids = f.read().split(',')
        
    prefix = 'https://files.rcsb.org/view'
    suffix = '.pdb'
    
    if not os.path.exists(savedir):
        os.makedirs(savedir)
    
    for i, id in enumerate(ids):
        pdb_id = id[:-1] + suffix
        url = os.path.join(prefix, pdb_id)
        path_name = os.path.join(savedir, pdb_id)
        urllib.request.urlretrieve(url, path_name)
        
    return None

In [ ]:
mode = ['test', 'train', 'valid']
pdbs = os.path.join(data_dir, 'pdbs')
id_dir = 'id_lists'
for mode in modes:
    download_pdbs(os.path.join(id_dir, mode + '.txt'), pdbs)

### Define functions for converting BioPDB structures into numpy arrays

In [115]:
from Bio.PDB import *
from Bio.Alphabet import ThreeLetterProtein

In [116]:
def relevant_atom(residue):
    '''Get the atom from a residue that is relevant for measuring residue-residue distances.
    In all amino acids this is CB, except for glycine where it is CA. If neither of these
    is present, return None'''
    if residue.get_resname().upper() == 'GLY':
        atom_id = 'CA'
    else:
        atom_id = 'CB'
    try:
        atom = residue[atom_id]
    except KeyError:
        atom = None
    return atom

def relevant_chain(structure, chain_id):
    '''Given a structure and a chain id, return that chain from the structure. 
    This is robust to upper and lower case.'''
    model = structure[0]
    try:
        chain = model[chain_id.upper()]
    except KeyError:
        try:
            chain = model[chain_id.lower()]
        except KeyError:
            raise ValueError("Structure {} has no chain {}. It only has {}".format(structure.get_id(), chain_id, list(structure.get_chains())))
    return chain

def get_seq_and_coords(structure, chain_id, alphabet, distances=False):
    '''Given a structure and chain id, return the sequence as a one-hot numpy
    array, and the coordinates of the residues as a numpy array. 
    Optionally, return the matrix of distances between the residues.'''
    chain = relevant_chain(structure, chain_id)
    
    residues = list(chain.get_residues())
    residues = [r for r in residues if r.get_resname() in alphabet]
    
    # sequence
    sequence = [r.get_resname() for r in residues]
    seq_vec = _string_vectorizer(sequence, alphabet)
    if not np.all(seq_vec.sum(1) == 1):
        bad_indices = np.where(seq_vec.sum(1) != 1)[0]
        bad_residues = [r.get_resname() for i, r in enumerate(residues) if i in bad_indices]
        out_str = "Sequence {}/{} has not been correctly one-hot encoded.\n".format(pdb_id, chain_id)
        out_str += "Indices {} in the sequence were incorrectly encoded\n".format(', '.join(bad_indices))
        our_str += "These correspond to the following residues: {}".format(', '.join(bad_residues))
        raise ValueError(out_str)
    
    # coords
    coords = np.zeros((len(residues), 3))
    for i, r1 in enumerate(residues):
        a1 = relevant_atom(r1)
        if a1 is not None:
            xyz = a1.get_coord()
            coords[i] = xyz
        else:
            coords[i] = [np.inf, np.inf, np.inf]
    
    # distances
    if distances:
        dists = np.zeros((len(residues), len(residues)))
        for i, r1 in enumerate(residues):
            a1 = relevant_atom(r1)
            for j, r2 in enumerate(residues):
                a2 = relevant_atom(r2)
                if a1 is not None and a2 is not None:
                    dists[i, j] = a1 - a2
                else:
                    dists[i, j] = np.inf
        return seq_vec, coords, dists
    return seq_vec, coords
            
def _string_vectorizer(strng, alphabet=string.ascii_uppercase):
    '''Given a string and alphabet, return a one-hot representation as a numpy array'''
    vector = np.array([[0 if char != letter else 1 for char in alphabet]
                  for letter in strng])
    return vector



### Define preprocessing function for a directory of pdb files

In [119]:
def preprocess_pdb_directory(pdb_directory, id_list_filename, file_format, n=None):
    '''Preprocess all of the pdb files in a directory that correspond to named ids in 
    a given id list. '''
    t = time.time()
    parent_dir, child_dir = os.path.split(pdb_directory)
    with open(id_list_filename, 'r') as f:
        ids = f.read().split(',')
                
    if file_format == 'pdb':
        parser = PDBParser()
    elif file_format == 'cif':
        parser = MMCIFParser()
    
    alphabet = [sym.upper() for sym in ThreeLetterProtein().letters]
    sequences = []
    coords = []
    
    for i, id in enumerate(ids):
            
        pdb_id = id[:-1]
        chain_id = id[-1]
        pdb_filename = pdb_id + '.' + file_format
        
        path_to_pdb = os.path.join(pdb_directory, pdb_filename)
        structure = parser.get_structure(pdb_id, path_to_pdb)
        
        seq_vec, coord = get_seq_and_coords(structure, chain_id, alphabet)
        
        sequences.append(seq_vec)
        coords.append(coord)
        
        if i > 0 and i % 500 == 0:
            logging.info("Preprocessed {} proteins".format(i))
        if n is not None and i == n - 1:
            break
    
    savedir = os.path.join(data_dir, 'preprocessed')
    if not os.path.exists(savedir):
        os.makedirs(savedir)
    save_filename = os.path.join(savedir, os.path.split(pdb_directory)[1] + '.pkl')
    logging.info("Preprocessed {} proteins in {:.1f} seconds".format(len(sequences), time.time() - t))
    with open(save_filename, 'wb') as f:
        pickle.dump(dict(sequences=sequences,coords=coords), f)
        
    return None
        
    

### Run preprocessing over the pdb directories we have downloaded

In [120]:
mode = ['test', 'train', 'valid']
for mode in modes:
    preprocess_pdb_directory(os.path.join(pdbs, mode), os.path.join(id_dir, mode + '.txt'), 'pdb')